# Further exploration of ipyautoui

**NOTE:** We will not export the content in this notebook to a Python module with nbdev.

### Goal of this notebook 

This notebook contains a few additional discussion and examples of using ipyautoui. One is to demonstrate that is relatively straightforward to modify the dashboard when built in a more modular way.

### Steps you will take in this notebook

1. Make the calculation and display of the smoothing optional.
2. Explore more of what `ipyautoui` can do.

## An extension: making the smoothing optional

It might be convenient for the smoothing to be optional, with settings for window size and order available if the user wants to have the smoothed line on their graph.

Our motivation for adding that feature is to illustrate the utility of pydantic and ipyautoui for rapidly refactoring the design of an interface.

### Step 1: make the smoothing settings a separate model

We begin by making a pydnatic model for just the smoothing settings. The fields `window_size` and `polynomial_order` are identical to what we defined in the previous notebook.

In [ ]:
from typing import Annotated

import ipywidgets as widgets
from ipyautoui import AutoUi
from pydantic import BaseModel, Field, model_validator

In [ ]:
class SmoothingSettings(BaseModel, validate_assignment=True):
    window_size: Annotated[int, Field(ge=2, le=100)] = 2
    polynomial_order: Annotated[int, Field(ge=1, le=10)] = 1

    # mode="after" means the validator runs after pydantic has checked that the individual
    # fields have values that are valid.
    @model_validator(mode="after")
    def limit_polynomial_order(self):
        
        if self.polynomial_order > self.window_size - 1:
            # Handle a bad polynomial order or window size
            raise ValueError("Polynomial order must be smaller than window size")
            
        # If we got this far the polynomial order is consistent with the window size
        # so return self. Failing to return self will end up causing an error.
        return self

Let's take a quick look at the ipyautoui-generated widget for this to make sure it is what we want.

In [ ]:
ui_smooth = AutoUi(SmoothingSettings)
ui_smooth

### Step 2: Change the model for the controls

Now we make a class to hold the controls which has two fields: `year_range` and `smoothing`. For `smoothing`, we use the union operator `|` in the type annotation to indicate that the `smoothing` can be either of type `SmoothingSettings` or can be `None`.

To define the `year_range` properly we use the `ConstrainedInt` type we defined in a previous notebook.

In [ ]:
from key.dashboard_pydantic.pydantic_model import ConstrainedInt

In [ ]:
class DataSelectorModel(BaseModel, validate_assignment=True):
    """
    Controls for temperature graph.

    👉👉👉 Replaces the DataSelectorModel in the original pydantic dashboard 👈👈👈
    """
    year_range: Annotated[
        # The key change is in the line below
        tuple[ConstrainedInt, ConstrainedInt],
        # With this change to the type we no longer need to tell ipyautoui
        # what kind of widget to use. Field contains just a brief description
        Field(description="Range of years to plot")
    ] = (1800, 2000)
    smoothing: SmoothingSettings | None

Next, let's see how ipyautoui renders this.

In [ ]:
controls = AutoUi(DataSelectorModel)
controls

### Step 3: Enforce the constraint between `window_size` and `polynomial_order`

In an earlier notebook we constrained the smoothing settings so that the user could not set an invalid value. We do that again below.

In [ ]:
from key.dashboard_pydantic.widgets_autoui import make_enforcer
controls.observe(make_enforcer(controls), "_value")

### Step 4: Revise the `DataAndPlot` class

Since we have made a change in the way that the settings are defined we need to update the class we made holding the table and graph.

The only new lines are indicated with this notation in the code below; the rest is unchanged from the earlier notebook.

```python
# 👉👉👉 Changes to the code are indicated by this 👈👈👈
```

In [ ]:
from ipydatagrid import DataGrid
from matplotlib import pyplot as plt
from scipy.signal import savgol_filter
import pandas as pd
import traitlets as tr

class DataAndPlot(tr.HasTraits):
    smoothing_info = tr.Dict(allow_none=True, default=None)
    
    def __init__(self):
        self.original_data = pd.read_csv(Path(DATA_DIR) / DATA_FILE, escapechar='#')
        self.plot_output = widgets.Output()
        self.data_output = DataGrid(self.original_data, header_visibility="column", auto_fit_columns=True)
        # 👉👉👉 New attribute to track whether smoothing is turned on   👈👈👈
        self.smooth = False

    @tr.observe('smoothing_info')
    def select_and_plot(self, change):
        # 👉👉👉 Set smoothing attribute based on value from controls 👈👈👈
        self.smooth = change["new"]["smoothing"] is not None

        # 👉👉👉 Only calculate smoothing column if smoothing is turned on 👈👈👈
        if self.smooth:
            self.window_size = change["new"]["smoothing"]["window_size"]
            self.polynomial_order = change["new"]["smoothing"]["polynomial_order"]
        
            self.original_data['Savitzky-Golay'] = savgol_filter(
                self.original_data['Temperature'], 
                self.window_size, 
                self.polynomial_order
            )
        self.year_range = change["new"]["year_range"]
        
        self.selected = (
            self.original_data[(self.original_data['Year'] >= self.year_range[0]) & 
                               (self.original_data['Year'] <= self.year_range[1])]
        )
        self.display_plot()
        self.display_data()

    def display_plot(self): 
        # This plotting function is copied from the first 
        # version of the dashboard.
        self.plot_output.clear_output(wait=True) 
        with self.plot_output: 
            plt.xlabel('Year') 
            plt.ylabel('Temperature Anomalies over Land w.r.t. 1951-80 (˚C)') 
            plt.title('Global Annual Mean Surface Air Temperature Change')
            plt.plot(self.selected['Year'], self.selected['Temperature'], label='Raw Data') 
            # 👉👉👉 Only plot smoothing if it is turned on 👈👈👈
            if self.smooth:
                plt.plot(self.selected['Year'], self.selected['Savitzky-Golay'], label='Smoothed Data') 
            plt.legend()
            plt.show() 

    def display_data(self):
        self.data_output.data = self.selected

### Step 6: build the dashboard

The code below is copy/pasted from `key/dashboard_pydantic/mail.py` with almost no modification. Only some import statements have been removed, and one line added to import the text boxes and some constants from the pydantic dashboard.

In [ ]:
# 👉👉👉 Get the unmodified widgets from the pydantic version of the dashboard 👈👈👈
from key.dashboard_pydantic.widgets_classes import TextBoxes, DATA_DIR, DATA_FILE

from pathlib import Path
# Create a VBox to hold the description and control widgets
desc_and_ctrl_box = widgets.VBox()

# Add a vertical box holding both table and plot visualizations of selected data
data_box = widgets.VBox()

# The entire widget
main_widget = widgets.HBox(children = (desc_and_ctrl_box, data_box))


data_and_plot = DataAndPlot()
text_boxes = TextBoxes()

data_accordion = widgets.Accordion(children=(data_and_plot.data_output,), titles=("Selected Data",))
desc_and_ctrl_box.children = (text_boxes, controls)
data_box.children = (data_accordion, data_and_plot.plot_output)


source = (controls, "_value")
target = (data_and_plot, "smoothing_info")
link = widgets.link(source, target)

for widget in controls.di_widgets.values():
    widget.layout.max_width = "250px"

In [ ]:
main_widget

## Try out the `ipyautoui` demo

The easiest way to get a better idea of what ipyautoui can do is to try out its demo. If you have time, feel free to explore!

In [ ]:
from ipyautoui import demo
demo()

## Anatomy of an AutoUi widget